In [1]:
import numpy as np
import pandas as pd

In [2]:
alldata = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\drop201904final.xlsx")
data1 = alldata.groupby(['LocationID']).mean()
data1 = data1[['Freq','pop_density','commercial_density','road_density','X','Y']]

from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(data1[['pop_density','commercial_density','road_density']])
data1[['pop_density','commercial_density','road_density']] = scaler.transform(data1[['pop_density','commercial_density','road_density']])

In [3]:
X = np.array(data1[["X"]])
Y = np.array(data1[["Y"]])
coords = np.hstack([X,Y])

y = np.array(data1[["Freq"]])

pop = np.array(data1[["pop_density"]])
com = np.array(data1[["commercial_density"]])
roa = np.array(data1[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.model import GWR

results1 = GWR(coords, y, x,49, kernel='bisquare').fit()

results1.R2

0.9132140047305656

In [11]:
coords = results1.coords
coords = pd.DataFrame(coords)

betas = results1.betas
betas = pd.DataFrame(betas)

tvalues = results1.tvalues
tvalues = pd.DataFrame(tvalues)

final_result = pd.concat([betas,tvalues,coords], axis = 1)
final_result.columns=['beta0','POP_beta','COM_beta','ROA_beta','t_beta0','t_POP_beta','t_COM_beta','t_ROA_beta','X','Y']

data2 = pd.merge(data1, final_result, left_on=['X','Y'], right_on=['X','Y'] , how='left')
data2.drop(['Freq','pop_density','commercial_density','road_density'],axis=1,inplace = True)
data2.to_csv(r'E:\\Capstone Project\\Data\\paper\\minor\\drop201904beta.csv',index=True,sep=',')

In [4]:
betas = results1.betas
betas = pd.DataFrame(betas)
betas.describe()

,0,1,2,3
count,258.000000,258.000000,258.000000,258.000000
mean,144.949210,38.168167,-5.089025,29.114552
std,236.519625,77.197735,47.081986,103.524852
min,-42.365384,-96.358800,-251.295069,-256.470127
25%,9.235516,-1.358937,-2.241750,-8.405099
50%,27.496854,3.310515,1.505590,1.185233
75%,105.391617,33.644181,12.940535,22.982650
max,857.210247,330.393276,125.263292,462.816067
